# Bài 3: Trả lời câu hỏi

Chào mừng bạn đến với bài tập thứ ba của khóa học 4. Trong bài tập này, bạn sẽ khám phá cách trả lời câu hỏi. Bạn sẽ triển khai "Chuyển văn bản sang văn bản từ Transformers" (hay còn gọi là T5). Vì bạn đã triển khai máy biến áp từ đầu vào tuần trước nên giờ đây bạn có thể sử dụng chúng.

<img src = "images/qa.png"> 


## Mục lục

- [Overview](#0-1)
- [Importing the Packages](#0-2)
- [1 - Prepare the data for pretraining T5](#1)
- [1.1 - Pre-Training Objective](#1-1)
- [1.2 - C4 Dataset](#1-2)
- [1.3 - Process C4](#1-3)
- [1.4 - Decode to Natural Language](#1-4)
- [1.5 - Tokenizing and Masking](#1-5)
- [Exercise 1 - tokenize_and_mask](#ex-1)
- [1.6 - Creating the Pairs](#1-6)
- [2 - Pretrain a T5 model using C4](#2)
- [2.1 - Instantiate a new transformer model](#2-1)
- [2.2 - C4 pretraining](#2-2)
- [3 - Fine tune the T5 model for Question Answering](#3)
- [3.1 - Creating a list of paired question and answers](#3-1)
- [Exercise 2 - Parse the SQuaD 2.0 dataset](#ex-2)
- [3.2 - Fine tune the T5 model](#3-2)
- [3.3 - Implement your Question Answering model](#3-3)
- [Exercise 3 - Implement the question answering function](#ex-3)

<a name='0-1'></a>
## Tổng quan

Nhiệm vụ này sẽ khác với hai nhiệm vụ trước. Do hạn chế về bộ nhớ của môi trường này và vì lý do thời gian, mô hình của bạn sẽ được đào tạo với các tập dữ liệu nhỏ, vì vậy bạn sẽ không nhận được các mô hình mà bạn có thể sử dụng trong sản xuất nhưng bạn sẽ có được kiến ​​thức cần thiết về cách hoạt động của các mô hình Ngôn ngữ Sáng tạo được đào tạo và sử dụng. Ngoài ra, bạn sẽ không dành quá nhiều thời gian cho kiến ​​trúc của các mô hình mà thay vào đó, bạn sẽ sử dụng một mô hình đã được đào tạo trước trên tập dữ liệu lớn hơn và tinh chỉnh nó để có kết quả tốt hơn.

Sau khi hoàn thành bài thí nghiệm này, bạn sẽ:
* Hiểu cách cấu trúc tập dữ liệu C4.
* Huấn luyện trước mô hình máy biến áp bằng Mô hình ngôn ngữ đeo mặt nạ.
* Hiểu cách hoạt động của mô hình "Chuyển văn bản sang văn bản từ Transformers" hoặc T5.
* Tinh chỉnh mô hình T5 để trả lời câu hỏi

Trước khi bắt đầu, hãy dành chút thời gian để đọc những lời khuyên sau:
#### MẸO ĐỂ ĐÁNH GIÁ THÀNH CÔNG BÀI VIẾT CỦA BẠN:
- Tất cả các ô đều bị đóng băng ngoại trừ những ô bạn cần gửi giải pháp của mình.
- Bạn có thể thêm các ô mới để thử nghiệm nhưng những ô này sẽ bị người chấm bỏ qua, vì vậy, đừng dựa vào các ô mới tạo để lưu trữ mã giải pháp của bạn, hãy sử dụng các vị trí được cung cấp cho việc này.
- Bạn có thể thêm nhận xét #grade-up-to-here vào bất kỳ ô đã chấm điểm nào để báo hiệu cho học sinh chấm điểm rằng học sinh chỉ được đánh giá đến điểm đó. Điều này rất hữu ích nếu bạn muốn kiểm tra xem mình có đang đi đúng hướng hay không ngay cả khi bạn chưa hoàn thành toàn bộ nhiệm vụ. Hãy nhớ xóa bình luận sau đó!
- Để gửi sổ ghi chép của bạn, hãy lưu nó rồi nhấp vào nút gửi màu xanh lam ở đầu trang.

<a name='0-2'></a>
## Nhập gói

Hãy bắt đầu bằng cách nhập tất cả các thư viện cần thiết.

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import traceback
import time
import json
from termcolor import colored
import string
import textwrap
import itertools
import numpy as np
import tensorflow_text as tf_text
import tensorflow as tf

import transformer_utils 
import utils

# Will come in handy later
wrapper = textwrap.TextWrapper(width=70)

# Set random seed
np.random.seed(42)

In [3]:
import w3_unittest

<a name='1'></a>
##1 - Chuẩn bị data cho pretraining T5

<a name='1-1'></a>
### 1.1 - Mục tiêu trước đào tạo

Trong giai đoạn đầu đào tạo mô hình T5 cho nhiệm vụ Trả lời câu hỏi, quy trình đào tạo trước bao gồm việc tận dụng mô hình ngôn ngữ đeo mặt nạ (MLM) trên một tập dữ liệu rất lớn, chẳng hạn như tập dữ liệu C4. Mục tiêu là cho phép mô hình học cách biểu diễn các từ và cụm từ theo ngữ cảnh, thúc đẩy sự hiểu biết sâu sắc hơn về ngữ nghĩa ngôn ngữ. Để bắt đầu đào tạo trước, điều cần thiết là phải sử dụng kiến ​​trúc Transformer, kiến ​​trúc này tạo thành xương sống của T5. Cơ chế tự chú ý của Transformer cho phép mô hình cân nhắc các phần khác nhau của chuỗi đầu vào một cách linh hoạt, nắm bắt các phần phụ thuộc tầm xa một cách hiệu quả.

Trước khi đi sâu vào đào tạo trước, việc xử lý trước dữ liệu kỹ lưỡng là rất quan trọng. Bộ dữ liệu C4, một bộ sưu tập các trang web đa dạng và phong phú, cung cấp nguồn phong phú cho các nhiệm vụ hiểu ngôn ngữ. Tập dữ liệu cần được mã hóa thành các đơn vị nhỏ hơn, chẳng hạn như từ phụ hoặc từ, để tạo điều kiện thuận lợi cho việc nhập mô hình. Ngoài ra, văn bản thường được phân đoạn thành các chuỗi hoặc lô có độ dài cố định, tối ưu hóa hiệu quả tính toán trong quá trình đào tạo.

Đối với mục tiêu lập mô hình ngôn ngữ được che giấu, một tỷ lệ phần trăm đầu vào được mã hóa được che giấu ngẫu nhiên và mô hình được đào tạo để dự đoán nội dung gốc của các mã thông báo được che giấu này. Quá trình này khuyến khích mô hình T5 nắm bắt các mối quan hệ theo ngữ cảnh giữa các từ và cụm từ, nâng cao khả năng tạo ra các phản hồi mạch lạc và phù hợp với ngữ cảnh trong các nhiệm vụ tiếp theo như trả lời câu hỏi.

Tóm lại, quá trình đào tạo trước mô hình T5 bao gồm việc sử dụng kiến ​​trúc Transformer trên tập dữ liệu lớn như C4, kết hợp với quá trình xử lý trước dữ liệu tỉ mỉ để chuyển đổi văn bản thô thành định dạng phù hợp cho việc đào tạo. Việc kết hợp mục tiêu mô hình hóa ngôn ngữ mặt nạ đảm bảo rằng mô hình học các cách biểu diễn theo ngữ cảnh mạnh mẽ, tạo nền tảng vững chắc cho việc tinh chỉnh tiếp theo đối với các tác vụ cụ thể như trả lời câu hỏi.

**Lưu ý:** Từ "mặt nạ" sẽ được sử dụng trong suốt bài tập này trong bối cảnh ẩn/xóa (các) từ

Bạn sẽ triển khai Mô hình ngôn ngữ đeo mặt nạ (MLM) như trong hình ảnh sau.

<img src = "images/loss.png" width="600" height = "400">

Giả sử bạn có văn bản sau: <span style="color:blue"> **Cảm ơn bạn <span style="color:red">vì đã mời </span> tôi đến bữa tiệc của bạn <span style="color:red" >tuần trước</span>** </span>


Bây giờ, khi nhập dữ liệu, bạn sẽ che các từ màu đỏ trong văn bản:

<span style="color:blue"> **Đầu vào:**</span> Cảm ơn bạn **X** tôi đã đến dự bữa tiệc **Y** của bạn trong tuần.

<span style="color:blue"">**Đầu ra:**</span> Mô hình sẽ dự đoán (các) từ cho **X** và **Y**.

**[EOS]** sẽ được sử dụng để đánh dấu sự kết thúc của chuỗi mục tiêu.

<a name='1-2'></a>
### 1.2 - Bộ dữ liệu C4

[C4 dataset](https://www.tensorflow.org/datasets/catalog/c4), còn được gọi là Common Crawl C4 (Common Crawl Corpus C4), là tập dữ liệu quy mô lớn về các trang web được thu thập bởi [C4 dataset](https://www.tensorflow.org/datasets/catalog/c4). Nó thường được sử dụng cho các nhiệm vụ xử lý ngôn ngữ tự nhiên khác nhau và nghiên cứu học máy. Mỗi mẫu trong tập dữ liệu C4 tuân theo một định dạng nhất quán, giúp mẫu này phù hợp với các mô hình đào tạo trước như BERT. Dưới đây là phần giải thích và mô tả ngắn gọn về tập dữ liệu C4:

- Định dạng: Mỗi mẫu trong tập dữ liệu C4 được biểu diễn dưới dạng một đối tượng JSON, chứa một số cặp khóa-giá trị.

- Nội dung: Trường “text” trong mỗi mẫu chứa nội dung văn bản thực tế được trích xuất từ ​​các trang web. Văn bản này thường bao gồm nhiều chủ đề và phong cách viết khác nhau nên đa dạng và phù hợp với các mô hình ngôn ngữ đào tạo.

- Siêu dữ liệu: Tập dữ liệu bao gồm siêu dữ liệu như 'độ dài nội dung', 'loại nội dung', 'dấu thời gian' và 'url', cung cấp thông tin bổ sung về từng trang web. 'Độ dài nội dung' chỉ định độ dài của nội dung, 'loại nội dung' mô tả loại nội dung (ví dụ: 'văn bản/thuần túy'), 'dấu thời gian' cho biết thời điểm trang web được thu thập thông tin và 'url' cung cấp nguồn URL của trang web.

- Ứng dụng: Bộ dữ liệu C4 được sử dụng phổ biến để huấn luyện và tinh chỉnh các mô hình ngôn ngữ quy mô lớn như BERT. Nó phục vụ như một nguồn tài nguyên có giá trị cho các nhiệm vụ như phân loại văn bản, nhận dạng thực thể được đặt tên, trả lời câu hỏi, v.v.

- Kích thước: Bộ dữ liệu C4 chứa hơn 800 GiB dữ liệu văn bản, phù hợp cho các mô hình đào tạo với hàng tỷ tham số.

Chạy ô bên dưới để xem tập dữ liệu C4 trông như thế nào.

In [4]:
# Load example jsons
with open('data/c4-en-10k.jsonl', 'r') as file:
    example_jsons = [json.loads(line.strip()) for line in file]

# Printing the examples to see how the data looks like
for i in range(5):
    print(f'example number {i+1}: \n\n{example_jsons[i]} \n')

example number 1: 

{'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.'} 

example number 2: 

{'text': 'Discussion in \'Mac OS X Lion (10.7)\' started by axboi87, Jan 20, 2012.\nI\'ve got a 500gb internal drive and a 240gb SSD.\nWhen trying to restore using disk utility i\'m given the err

<a name='1-3'></a>
### 1.3 - Quy trình C4

Với mục đích lấy trước mẫu T5 các bạn sẽ chỉ sử dụng `nội dung` của mỗi mục. Trong đoạn mã sau, bạn chỉ lọc trường `văn bản` từ tất cả các mục trong tập dữ liệu. Đây là dữ liệu bạn sẽ sử dụng để tạo `đầu vào` và `đích` cho mô hình ngôn ngữ của mình.

In [5]:
# Grab text field from dictionary
natural_language_texts = [example_json['text'] for example_json in example_jsons]

# Print the first text example
print(natural_language_texts[0])

Beginners BBQ Class Taking Place in Missoula!
Do you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.
He will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.
The cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.


<a name='1-4'></a>
### 1.4 - Giải mã sang ngôn ngữ tự nhiên

[SentencePieceTokenizer](https://www.tensorflow.org/text/api_docs/python/text/SentencepieceTokenizer), được sử dụng trong đoạn mã, mã hóa văn bản thành các đơn vị từ phụ, nâng cao khả năng xử lý các cấu trúc từ phức tạp, các từ không có từ vựng và hỗ trợ đa ngôn ngữ. Nó đơn giản hóa quá trình tiền xử lý, đảm bảo mã thông báo nhất quán và tích hợp liền mạch với các khung máy học.

Trong tác vụ này, một mô hình SentencePiece được tải từ một tệp, được sử dụng để mã hóa văn bản thành các từ phụ được biểu thị bằng ID số nguyên.

In [6]:
# Special tokens
# PAD, EOS = 0, 1

with open("./models/sentencepiece.model", "rb") as f:
    pre_trained_tokenizer = f.read()
    
tokenizer = tf_text.SentencepieceTokenizer(pre_trained_tokenizer, out_type=tf.int32)

Trong mã thông báo này, chuỗi `</s>` được sử dụng làm mã thông báo `EOS`. Theo mặc định, tokenizer không thêm `EOS` vào cuối mỗi câu, vì vậy bạn cần thêm thủ công khi được yêu cầu. Hãy xác minh id nào tương ứng với mã thông báo này:

In [7]:
eos = tokenizer.string_to_id("</s>").numpy()

print("EOS: " + str(eos))

EOS: 1


Mã này hiển thị quá trình mã hóa các từ riêng lẻ từ một văn bản nhất định, trong trường hợp này là mục nhập đầu tiên của tập dữ liệu.

In [8]:
# printing the encoding of each word to see how subwords are tokenized
tokenized_text = [(list(tokenizer.tokenize(word).numpy()), word) for word in natural_language_texts[2].split()]

print("Word\t\t-->\tTokenization")
print("-"*40)
for element in tokenized_text:
    print(f"{element[1]:<8}\t-->\t{element[0]}")

Word		-->	Tokenization
----------------------------------------
Foil    	-->	[4452, 173]
plaid   	-->	[30772]
lycra   	-->	[3, 120, 2935]
and     	-->	[11]
spandex 	-->	[8438, 26, 994]
shortall	-->	[710, 1748]
with    	-->	[28]
metallic	-->	[18813]
slinky  	-->	[3, 7, 4907, 63]
insets. 	-->	[16, 2244, 7, 5]
Attached	-->	[28416, 15, 26]
metallic	-->	[18813]
elastic 	-->	[15855]
belt    	-->	[6782]
with    	-->	[28]
O-ring. 	-->	[411, 18, 1007, 5]
Headband	-->	[3642, 3348]
included.	-->	[1285, 5]
Great   	-->	[1651]
hip     	-->	[5436]
hop     	-->	[13652]
or      	-->	[42]
jazz    	-->	[9948]
dance   	-->	[2595]
costume.	-->	[11594, 5]
Made    	-->	[6465]
in      	-->	[16]
the     	-->	[8]
USA.    	-->	[2312, 5]


Và như thường lệ, thư viện cung cấp chức năng biến mã thông báo số thành văn bản mà con người có thể đọc được. Hãy nhìn cách nó hoạt động.

In [9]:
# We can see that detokenize successfully undoes the tokenization
print(f"tokenized: {tokenizer.tokenize('Beginners')}\ndetokenized: {tokenizer.detokenize(tokenizer.tokenize('Beginners'))}")

tokenized: [12847   277]
detokenized: b'Beginners'


Như bạn có thể thấy ở trên, bạn có thể lấy một đoạn chuỗi và mã hóa nó.

Bây giờ bạn sẽ tạo các cặp `input` và `target` cho phép bạn huấn luyện mô hình của mình. T5 sử dụng các id ở cuối tệp vocab làm trọng điểm. Ví dụ: nó sẽ thay thế:
- `vocab_size - 1` bởi `<Z>`
- `vocab_size - 2` bởi `<Y>`
- và kể từ đó trở đi.

Nó gán cho mỗi từ một `chr`.

Hàm `pretty_decode` bên dưới mà bạn sẽ sử dụng sau đây sẽ giúp xử lý loại khi giải mã. Hãy xem và cố gắng hiểu chức năng này đang làm gì.


Thông báo rằng:
```python
string.ascii_letters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
```

**LƯU Ý:** Các mục tiêu có thể có nhiều hơn 52 lính canh mà chúng tôi thay thế, nhưng đây chỉ là để giúp bạn hình dung về mọi thứ.

In [10]:
def get_sentinels(tokenizer, display=False):
    sentinels = {}
    vocab_size = tokenizer.vocab_size(name=None)
    for i, char in enumerate(reversed(string.ascii_letters), 1):
        decoded_text = tokenizer.detokenize([vocab_size - i]).numpy().decode("utf-8")
        
        # Sentinels, ex: <Z> - <a>
        sentinels[decoded_text] = f'<{char}>'    
    
        if display:
            print(f'The sentinel is <{char}> and the decoded token is:', decoded_text)

    return sentinels

def pretty_decode(encoded_str_list, sentinels, tokenizer):
    # If already a string, just do the replacements.
    if tf.is_tensor(encoded_str_list) and encoded_str_list.dtype == tf.string:
        for token, char in sentinels.items():
            encoded_str_list = tf.strings.regex_replace(encoded_str_list, token, char)
        return encoded_str_list
  
    # We need to decode and then prettyfy it.
    return pretty_decode(tokenizer.detokenize(encoded_str_list), sentinels, tokenizer)

In [11]:
sentinels = get_sentinels(tokenizer, display=True)

The sentinel is <Z> and the decoded token is: Internațional
The sentinel is <Y> and the decoded token is: erwachsene
The sentinel is <X> and the decoded token is: Cushion
The sentinel is <W> and the decoded token is: imunitar
The sentinel is <V> and the decoded token is: Intellectual
The sentinel is <U> and the decoded token is: traditi
The sentinel is <T> and the decoded token is: disguise
The sentinel is <S> and the decoded token is: exerce
The sentinel is <R> and the decoded token is: nourishe
The sentinel is <Q> and the decoded token is: predominant
The sentinel is <P> and the decoded token is: amitié
The sentinel is <O> and the decoded token is: erkennt
The sentinel is <N> and the decoded token is: dimension
The sentinel is <M> and the decoded token is: inférieur
The sentinel is <L> and the decoded token is: refugi
The sentinel is <K> and the decoded token is: cheddar
The sentinel is <J> and the decoded token is: unterlieg
The sentinel is <I> and the decoded token is: garanteaz
Th

Bây giờ, hãy sử dụng hàm `pretty_decode` trong câu sau. Lưu ý rằng tất cả các từ được liệt kê là trọng điểm, sẽ được thay thế bằng hàm có trọng điểm tương ứng. Đó có thể là một nhược điểm của phương pháp này, nhưng bây giờ đừng lo lắng về nó.

In [12]:
pretty_decode(tf.constant("I want to dress up as an Intellectual this halloween."), sentinels, tokenizer)

<tf.Tensor: shape=(), dtype=string, numpy=b'I want to dress up as an <V> this <b>.'>

Các hàm trên làm cho `đầu vào` và `đích` của bạn dễ đọc hơn. Ví dụ: bạn có thể thấy nội dung như thế này khi triển khai chức năng tạo mặt nạ bên dưới.

- <span style="color:red"> Câu đầu vào: </span> Younes và Lukasz đã cùng làm việc trong phòng thí nghiệm sau bữa trưa ngày hôm qua.
- <span style="color:red">Đầu vào: </span> Younes và Lukasz **Z** cùng nhau ở **Y** ngày hôm qua sau bữa trưa.
- <span style="color:red">Mục tiêu: </span> **Z** đang làm việc tại phòng thí nghiệm **Y**.


<a name='1-5'></a>
### 1.5 - Token hóa và che giấu

Trong nhiệm vụ này, bạn sẽ triển khai hàm `tokenize_and_mask`, hàm này mã hóa và che dấu các từ đầu vào dựa trên một xác suất nhất định. Xác suất được kiểm soát bởi tham số `noise`, thường được đặt để che khoảng `15%` số từ trong văn bản đầu vào. Hàm sẽ tạo hai danh sách các chuỗi được mã hóa theo thuật toán được nêu bên dưới:

<a name='ex-1'></a>
### Bài tập 1 - tokenize_and_mask

- Bắt đầu với hai danh sách trống: `inps` và `targs`
- Mã hóa văn bản đầu vào bằng cách sử dụng mã thông báo đã cho.
- Đối với mỗi `token` trong chuỗi token hóa:
- Tạo số ngẫu nhiên (mô phỏng việc tung đồng xu có trọng số)
- Nếu giá trị ngẫu nhiên lớn hơn ngưỡng cho trước(nhiễu):
- Thêm token hiện tại vào danh sách `inps`
- Khác:
- Nếu phải thêm lính canh mới (đọc ghi chú **):
- Tính toán ID trọng điểm tiếp theo bằng cách sử dụng tiến trình.
- Thêm canh gác vào `inps` và `targs` để đánh dấu vị trí của phần tử bị che.
- Thêm token hiện tại vào danh sách `targs`.

** Có một trường hợp đặc biệt cần xem xét. Nếu hai hoặc nhiều mã thông báo liên tiếp bị che trong quá trình này, bạn không cần thêm trọng điểm mới vào chuỗi. Để giải quyết vấn đề này, hãy sử dụng cờ `prev_no_mask`, bắt đầu là `True` nhưng được chuyển thành `False` mỗi khi bạn ẩn một phần tử mới. Mã thêm điểm canh gác sẽ chỉ được thực thi nếu trước khi ẩn mã thông báo, cờ ở trạng thái `True`.


In [16]:
# GRADED FUNCTION: tokenize_and_mask
def tokenize_and_mask(text, 
                      noise=0.15, 
                      randomizer=np.random.uniform, 
                      tokenizer=None):
    """Tokenizes and masks a given input.

    Args:
        text (str or bytes): Text input.
        noise (float, optional): Probability of masking a token. Defaults to 0.15.
        randomizer (function, optional): Function that generates random values. Defaults to np.random.uniform.
        tokenizer (function, optional): Tokenizer function. Defaults to tokenize.

    Returns:
        inps, targs: Lists of integers associated to inputs and targets.
    """
    
    # Current sentinel number (starts at 0)
    cur_sentinel_num = 0
    
    # Inputs and targets
    inps, targs = [], []

    # Vocab_size
    vocab_size = int(tokenizer.vocab_size())
    
    # EOS token id 
    # Must be at the end of each target!
    eos = tokenizer.string_to_id("</s>").numpy()
    
    ### START CODE HERE ###
    
    # prev_no_mask is True if the previous token was NOT masked, False otherwise
    # set prev_no_mask to True
    prev_no_mask = True
    
    # Loop over the tokenized text
    for token in tokenizer.tokenize(text).numpy():
        
        # Generate a random value between 0 and 1
        rnd_val = randomizer() 
        
        # Check if the noise is greater than a random value (weighted coin flip)
        if noise > rnd_val:
            
            # Check if previous token was NOT masked
            if prev_no_mask:
                
                # Current sentinel increases by 1
                cur_sentinel_num += 1
                
                # Compute end_id by subtracting current sentinel value out of the total vocabulary size
                end_id = vocab_size - cur_sentinel_num
                
                # Append end_id at the end of the targets
                targs.append(end_id)
                
                # Append end_id at the end of the inputs
                inps.append(end_id)
                
            # Append token at the end of the targets
            targs.append(token)
            
            # set prev_no_mask accordingly
            prev_no_mask = False

        else:
            
            # Append token at the end of the inputs
            inps.append(token)
            
            # Set prev_no_mask accordingly
            prev_no_mask = True
    
    
    # Add EOS token to the end of the targets
    targs.append(eos)
    
    ### END CODE HERE ###
    
    return inps, targs

In [17]:
# Some logic to mock a np.random value generator
# Needs to be in the same cell for it to always generate same output
def testing_rnd():
    def dummy_generator():
        vals = np.linspace(0, 1, 10)
        cyclic_vals = itertools.cycle(vals)
        for _ in range(100):
            yield next(cyclic_vals)

    dumr = itertools.cycle(dummy_generator())

    def dummy_randomizer():
        return next(dumr)
    
    return dummy_randomizer

input_str = 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers.'

inps, targs = tokenize_and_mask(input_str, randomizer=testing_rnd(), tokenizer=tokenizer)
print(f"tokenized inputs - shape={len(inps)}:\n\n{inps}\n\ntargets - shape={len(targs)}:\n\n{targs}")

tokenized inputs - shape=53:

[31999, 15068, 4501, 3, 12297, 3399, 16, 5964, 7115, 31998, 531, 25, 241, 12, 129, 394, 44, 492, 31997, 58, 148, 56, 43, 8, 1004, 6, 474, 31996, 39, 4793, 230, 5, 2721, 6, 1600, 1630, 31995, 1150, 4501, 15068, 16127, 6, 9137, 2659, 5595, 31994, 782, 3624, 14627, 15, 12612, 277, 5]

targets - shape=19:

[31999, 12847, 277, 31998, 9, 55, 31997, 3326, 15068, 31996, 48, 30, 31995, 727, 1715, 31994, 45, 301, 1]


#### **Đầu ra dự kiến:**
```
tokenized inputs - shape=53:

[31999 15068  4501     3 12297  3399    16  5964  7115 31998   531    25
   241    12   129   394    44   492 31997    58   148    56    43     8
  1004     6   474 31996    39  4793   230     5  2721     6  1600  1630
 31995  1150  4501 15068 16127     6  9137  2659  5595 31994   782  3624
 14627    15 12612   277     5]

targets - shape=19:

[31999 12847   277 31998     9    55 31997  3326 15068 31996    48    30
 31995   727  1715 31994    45   301     1]
```

In [18]:
# Test your implementation!
w3_unittest.test_tokenize_and_mask(tokenize_and_mask)

 All tests passed


Bây giờ bạn sẽ sử dụng thông tin đầu vào và mục tiêu từ hàm `tokenize_and_mask` mà bạn đã triển khai ở trên. Hãy xem phiên bản được giải mã của câu bị che bằng cách sử dụng `inps` và `targs` từ câu trên.

In [19]:
print('Inputs: \n\n', pretty_decode(inps, sentinels, tokenizer).numpy())
print('\nTargets: \n\n', pretty_decode(targs, sentinels, tokenizer).numpy())

Inputs: 

 b'<Z> BBQ Class Taking Place in Missoul <Y> Do you want to get better at making <X>? You will have the opportunity, put <W> your calendar now. Thursday, September 22 <V> World Class BBQ Champion, Tony Balay <U>onestar Smoke Rangers.'

Targets: 

 b'<Z> Beginners <Y>a! <X> delicious BBQ <W> this on <V>nd join <U> from L'


<a name='1-6'></a>
### 1.6 - Tạo cặp

Bây giờ bạn sẽ tạo các cặp bằng cách sử dụng tập dữ liệu của mình. Bạn sẽ lặp lại dữ liệu của mình và tạo các cặp (inp, targ) bằng cách sử dụng các hàm mà chúng tôi đã cung cấp cho bạn.

In [20]:
# Apply tokenize_and_mask
inputs_targets_pairs = [tokenize_and_mask(text.encode('utf-8', errors='ignore').decode('utf-8'), tokenizer=tokenizer) 
                        for text in natural_language_texts[0:2000]]

In [21]:
def display_input_target_pairs(inputs_targets_pairs, sentinels, wrapper=textwrap.TextWrapper(width=70), tokenizer=tokenizer):
    for i, inp_tgt_pair in enumerate(inputs_targets_pairs, 1):
        inps, tgts = inp_tgt_pair
        inps = str(pretty_decode(inps, sentinels, tokenizer).numpy(), encoding='utf-8')
        tgts = str(pretty_decode(tgts, sentinels, tokenizer).numpy(), encoding='utf-8')
        print(f'[{i}]\n\n'
              f'inputs:\n{wrapper.fill(text=inps)}\n\n'
              f'targets:\n{wrapper.fill(text=tgts)}\n\n\n')

# Print 3 samples. We print inputs with less than 100 tokens. It is just to give you and idea of the process
display_input_target_pairs(filter(lambda x: len(x[0]) < 100, inputs_targets_pairs[0:12]), sentinels, wrapper, tokenizer)

[1]

inputs:
<Z>il plaid <Y>lycra <X> spandex shortall with metallic slinky
<W>sets. Attache <V> metallic elastic belt with O <U>ring. Head <T>
included. Great hip hop<S> jazz dance costume.<R> in the USA.

targets:
<Z> Fo <Y>  <X> and <W> in <V>d <U>- <T>band<S> or<R> Made



[2]

inputs:
I thought I was going to <Z> 3rd season <Y> Wire tonight. <X> there
was a commentary <W> 11, so I had to re <V>watch <U> Ground with <T>
commentary. Hopefully<S> can finish<R> season <Q>.

targets:
<Z> finish the <Y> of the <X> But <W> on episode <V>- <U> Middle <T>
the<S> I<R> the <Q> next weekend



[3]

inputs:
Pencarian <Z>FILM Untuk " <Y>eace <X>er 2017 <W> yuk mampir ke channel
say <V>. Edges <U> provides the l.. A corrupt cop makes one w.. <T>er
2017  ⁇ <S> ⁇  .. Náo Lo ⁇ n - Peace Break.. Please subscribe and hit
..<R> in HD at http://.. <Q> cannot believe I manage..

targets:
<Z>  <Y>P <X> Break <W>" <V>. <U> East <T> Peace Break<S> <R> uploaded
<Q> I





<a name='2'></a>
##2 - Pretrain mẫu T5 sử dụng C4

Bây giờ bạn sẽ sử dụng kiến ​​trúc của Transformer mà bạn đã mã hóa trong bài tập trước để tóm tắt văn bản, nhưng lần này là để trả lời các câu hỏi. Thay vì đào tạo mô hình trả lời câu hỏi từ đầu, trước tiên bạn sẽ “huấn luyện trước” mô hình bằng tập dữ liệu C4 vừa xử lý. Điều này sẽ giúp mô hình tìm hiểu cấu trúc chung của ngôn ngữ từ một tập dữ liệu lớn. Điều này dễ thực hiện hơn nhiều vì bạn không cần gắn nhãn cho bất kỳ dữ liệu nào mà chỉ cần sử dụng mặt nạ, việc này được thực hiện tự động. Sau đó, bạn sẽ sử dụng dữ liệu từ bộ SQuAD để hướng dẫn mô hình trả lời các câu hỏi theo ngữ cảnh. Để bắt đầu, chúng ta hãy xem lại kiến ​​trúc của Transformer.

<img src = "images/fulltransformer.png" width="300" height="600">

<a name='2-1'></a>
### 2.1 - Khởi tạo mô hình máy biến áp mới

Chúng tôi đã đóng gói mã được triển khai vào tuần trước vào tệp `Transformer.py`. Bạn có thể nhập nó vào đây và thiết lập với cùng cấu hình được sử dụng ở đó.

In [22]:
# Define the model parameters
num_layers = 2
embedding_dim = 128
fully_connected_dim = 128
num_heads = 2
positional_encoding_length = 256

encoder_vocab_size = int(tokenizer.vocab_size())
decoder_vocab_size = encoder_vocab_size

# Initialize the model
transformer = transformer_utils.Transformer(
    num_layers, 
    embedding_dim, 
    num_heads, 
    fully_connected_dim,
    encoder_vocab_size, 
    decoder_vocab_size, 
    positional_encoding_length, 
    positional_encoding_length,
)

Bây giờ, bạn sẽ xác định trình tối ưu hóa và hàm mất mát. Đối với nhiệm vụ này, mô hình sẽ cố gắng dự đoán các từ bị che, do đó, giống như trong bài thực hành trước, hàm mất sẽ là `SparseCategoricalCrossEntropy`.

In [23]:
learning_rate = transformer_utils.CustomSchedule(embedding_dim)
optimizer = tf.keras.optimizers.Adam(0.0001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
train_loss = tf.keras.metrics.Mean(name='train_loss')

# Here you will store the losses, so you can later plot them
losses = []

<a name='2-2'></a>
### 2.2 - Cch đào tạo lại

Để đào tạo mô hình Tensorflow, bạn cần sắp xếp dữ liệu thành các tập dữ liệu. Bây giờ, bạn sẽ nhận được `đầu vào` và `mục tiêu` cho mô hình máy biến áp từ `inputs_targets_pairs`. Trước khi tạo tập dữ liệu, bạn cần đảm bảo rằng tất cả `đầu vào` đều có cùng độ dài bằng cách cắt bớt các chuỗi dài hơn và đệm các chuỗi ngắn hơn bằng `0`. Điều tương tự phải được thực hiện cho các mục tiêu. Hàm `tf.keras.preprocessing.sequence.pad_sequences` sẽ giúp bạn ở đây, như trong bài tập tuần trước.

Bạn sẽ sử dụng `BATCH_SIZE = 64`

In [25]:
# Limit the size of the input and output data so this can run in this environment
encoder_maxlen = 150
decoder_maxlen = 50

inputs = tf.keras.preprocessing.sequence.pad_sequences([x[0] for x in inputs_targets_pairs], maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences([x[1] for x in inputs_targets_pairs], maxlen=decoder_maxlen, padding='post', truncating='post')

inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

# Create the final training dataset.
BUFFER_SIZE = 10000
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Bây giờ, bạn có thể chạy vòng huấn luyện trong 10 kỷ nguyên. Việc chạy nó với tập dữ liệu lớn như C4 trên một máy tính tốt có đủ bộ nhớ và GPU tốt có thể mất hơn 24 giờ. Tại đây, bạn sẽ chạy một số kỷ nguyên bằng cách sử dụng một phần nhỏ của tập dữ liệu C4 để minh họa. Sẽ chỉ mất vài phút nhưng mô hình sẽ không mạnh mẽ lắm.

In [ ]:
# Define the number of epochs
epochs = 10

# Training loop
for epoch in range(epochs):
    
    start = time.time()
    train_loss.reset_states()
    number_of_batches=len(list(enumerate(dataset)))

    for (batch, (inp, tar)) in enumerate(dataset):
        print(f'Epoch {epoch+1}, Batch {batch+1}/{number_of_batches}', end='\r')
        transformer_utils.train_step(inp, tar, transformer, loss_object, optimizer, train_loss)
    
    print (f'Epoch {epoch+1}, Loss {train_loss.result():.4f}')
    losses.append(train_loss.result())
    
    print (f'Time taken for one epoch: {time.time() - start} sec')

# Save the pretrained model
# transformer.save_weights('./model_c4_temp')

**Tải mô hình đã được huấn luyện trước**

Để cho thấy mô hình này thực sự mạnh mẽ đến mức nào, chúng tôi đã đào tạo mô hình này trong nhiều kỷ nguyên với tập dữ liệu đầy đủ trong Colab và lưu trọng số cho bạn. Bạn có thể tải chúng bằng cách sử dụng ô bên dưới. Đối với phần còn lại của cuốn sổ, bạn sẽ thấy được sức mạnh của việc học chuyển giao trong thực tế.

In [26]:
transformer.load_weights('./pretrained_models/model_c4')

<a name='3'></a>
## 3. Tinh chỉnh mẫu T5 để trả lời câu hỏi

Bây giờ, bạn sẽ tinh chỉnh mô hình được huấn luyện trước để Trả lời câu hỏi bằng cách sử dụng [SQUad 2.0 dataset](https://rajpurkar.github.io/SQuAD-explorer/).

SQuAD, viết tắt của Bộ dữ liệu trả lời câu hỏi Stanford, là bộ dữ liệu được thiết kế để đào tạo và đánh giá các hệ thống trả lời câu hỏi. Nó bao gồm các câu hỏi thực tế do con người đặt ra trên một tập hợp các bài viết Wikipedia, trong đó câu trả lời cho mỗi câu hỏi là một đoạn văn bản cụ thể trong bài viết tương ứng.

SQuAD 1.1, phiên bản trước của bộ dữ liệu SQuAD, chứa hơn 100.000 cặp câu hỏi-câu trả lời trên khoảng 500 bài viết.
SQuAD 2.0, chứa thêm 50.000 câu hỏi không cần trả lời. Bộ câu hỏi bổ sung này có thể giúp đào tạo các mô hình để phát hiện những câu hỏi không thể trả lời.

Hãy tải tập dữ liệu.

In [27]:
with open('data/train-v2.0.json', 'r') as f:
    example_jsons = json.load(f)

example_jsons = example_jsons['data']

print('Number of articles: ' + str(len(example_jsons)))

Number of articles: 442


Cấu trúc của mỗi bài viết như sau:
- `title`: Tiêu đề bài viết
- `paragraphs`: Danh sách các đoạn văn và câu hỏi liên quan đến chúng
- `ngữ cảnh`: Văn bản đoạn văn thực tế
- `was`: Bộ câu hỏi liên quan đến đoạn văn
- `câu hỏi`: Một câu hỏi
- `is`: Mã định danh duy nhất của câu hỏi
- `is_imposible`: Boolean, chỉ định câu hỏi có thể được trả lời hay không
- `answers`: Tập hợp các câu trả lời có thể có cho câu hỏi
- `text`: Câu trả lời
- `answer_start`: Chỉ mục của ký tự bắt đầu câu chứa câu trả lời rõ ràng cho câu hỏi

Hãy xem một bài viết bằng cách chạy ô tiếp theo. Lưu ý rằng `ngữ cảnh` thường là thành phần cuối cùng của mỗi đoạn văn:

In [28]:
example_article = example_jsons[0]
example_article

print("Title: " + example_article["title"])
print(example_article["paragraphs"][0])

Title: Beyoncé
{'qas': [{'question': 'When did Beyonce start becoming popular?', 'id': '56be85543aeaaa14008c9063', 'answers': [{'text': 'in the late 1990s', 'answer_start': 269}], 'is_impossible': False}, {'question': 'What areas did Beyonce compete in when she was growing up?', 'id': '56be85543aeaaa14008c9065', 'answers': [{'text': 'singing and dancing', 'answer_start': 207}], 'is_impossible': False}, {'question': "When did Beyonce leave Destiny's Child and become a solo singer?", 'id': '56be85543aeaaa14008c9066', 'answers': [{'text': '2003', 'answer_start': 526}], 'is_impossible': False}, {'question': 'In what city and state did Beyonce  grow up? ', 'id': '56bf6b0f3aeaaa14008c9601', 'answers': [{'text': 'Houston, Texas', 'answer_start': 166}], 'is_impossible': False}, {'question': 'In which decade did Beyonce become famous?', 'id': '56bf6b0f3aeaaa14008c9602', 'answers': [{'text': 'late 1990s', 'answer_start': 276}], 'is_impossible': False}, {'question': 'In what R&B group was she the

Bài viết trước có thể khó điều hướng nên đây là một đoạn ví dụ được định dạng đẹp mắt:
```python
{
  "context": "Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles 'Crazy in Love' and 'Baby Boy'",
  "qas": [
    {
      "question": "When did Beyonce start becoming popular?",
      "id": "56be85543aeaaa14008c9063",
      "answers": [
        {
          "text": "in the late 1990s",
          "answer_start": 269
        }
      ],
      "is_impossible": false
    },
    {
      "question": "What areas did Beyonce compete in when she was growing up?",
      "id": "56be85543aeaaa14008c9065",
      "answers": [
        {
          "text": "singing and dancing",
          "answer_start": 207
        }
      ],
      "is_impossible": false
    }
  ]
}
```

<a name='3-1'></a>
### 3.1 - Tạo danh sách câu hỏi và câu trả lời ghép đôi

Bạn được giao nhiệm vụ tạo các cặp đầu vào/đầu ra cho mô hình Trả lời câu hỏi (QA) bằng cách sử dụng bộ dữ liệu SQuAD 2.0. Mỗi cặp theo cấu trúc:

- đầu vào: `câu hỏi: <Q> bối cảnh: <P>`
- mục tiêu: `trả lời: <A>`

Ở đây, `<Q>` thể hiện câu hỏi trong ngữ cảnh của đoạn văn đã cho `<P>`, và `<A>` là một câu trả lời khả dĩ.

Trong cuốn sổ tay này, chúng ta sẽ tập trung vào một câu trả lời duy nhất cho mỗi câu hỏi. Tuy nhiên, điều cần lưu ý là tập dữ liệu chứa các câu hỏi có nhiều câu trả lời. Khi đào tạo một mô hình trong các tình huống thực tế, hãy cân nhắc việc đưa vào tất cả thông tin có sẵn.

<a name='ex-2'></a>
### Bài tập 2 - Phân tích bộ dữ liệu SQuaD 2.0

Nhiệm vụ của bạn là triển khai hàm pars_squad, hàm này lặp lại tất cả các bài viết, đoạn văn và câu hỏi trong tập dữ liệu SQuAD. Trích xuất các cặp đầu vào và mục tiêu cho mô hình QA bằng mẫu mã được cung cấp.
- Bắt đầu với hai danh sách trống: `inputs` và `target`.
- Lặp lại tất cả các bài viết trong tập dữ liệu.
- Đối với mỗi bài viết, lặp lại từng đoạn văn.
- Trích xuất ngữ cảnh trong đoạn văn.
- Lặp lại từng câu hỏi trong đoạn văn đã cho.
- Kiểm tra xem câu hỏi có phải là không thể thực hiện được và có ít nhất một câu trả lời hay không.
- Nếu đáp ứng điều kiện trên thì tạo chuỗi `question_context` như mô tả trong cấu trúc đầu vào.
- Tạo chuỗi `answer` bằng cách sử dụng câu trả lời đầu tiên trong số các câu trả lời có sẵn.
- Nối `question_context` vào danh sách `inputs`.
- Nối `answer` vào danh sách `target`.

In [33]:
# GRADED FUNCTION: parse_squad
def parse_squad(dataset):
    """Extract all the answers/questions pairs from the SQuAD dataset

    Args:
        dataset (dict): The imported JSON dataset

    Returns:
        inputs, targets: Two lists containing the inputs and the targets for the QA model
    """

    inputs, targets = [], []

    ### START CODE HERE ###
    
    # Loop over all the articles
    for article in dataset:
        
        # Loop over each paragraph of each article
        for paragraph in article['paragraphs']:
            
            # Extract context from the paragraph
            context = paragraph['context']
            
            #Loop over each question of the given paragraph
            for qa in paragraph['qas']:
                
                # If this question is not impossible and there is at least one answer
                if len(qa['answers']) > 0 and not(qa['is_impossible']):
                    
                    # Create the question/context sequence
                    question_context = 'question: ' + qa['question'] + ' context: ' + context
                    
                    # Create the answer sequence. Use the text field of the first answer
                    answer = 'answer: ' + qa['answers'][0]['text']
                    
                    # Add the question_context to the inputs list
                    inputs.append(question_context)
                    
                    # Add the answer to the targets list
                    targets.append(answer)
    
    ### END CODE HERE ###
    
    return inputs, targets

In [34]:
inputs, targets =  parse_squad(example_jsons)          
print("Number of question/answer pairs: " + str(len(inputs)))

print('\nFirst Q/A pair:\n\ninputs: ' + colored(inputs[0], 'blue'))
print('\ntargets: ' + colored(targets[0], 'green'))
print('\nLast Q/A pair:\n\ninputs: ' + colored(inputs[-1], 'blue'))
print('\ntargets: ' + colored(targets[-1], 'green'))

Number of question/answer pairs: 86821

First Q/A pair:

inputs: question: When did Beyonce start becoming popular? context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

targets: answer: in the late 1990s

Last Q/A pair:

inputs: question: What is KMC an initialism of? context: Kathmandu Metropolitan City (KMC), in order to promote international r

#### **Đầu ra dự kiến:**
```
Number of question/answer pairs: 86821

First Q/A pair:

inputs: question: When did Beyonce start becoming popular? context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

targets: answer: in the late 1990s

Last Q/A pair:

inputs: question: What is KMC an initialism of? context: Kathmandu Metropolitan City (KMC), in order to promote international relations has established an International Relations Secretariat (IRC). KMC's first international relationship was established in 1975 with the city of Eugene, Oregon, United States. This activity has been further enhanced by establishing formal relationships with 8 other cities: Motsumoto City of Japan, Rochester of the USA, Yangon (formerly Rangoon) of Myanmar, Xi'an of the People's Republic of China, Minsk of Belarus, and Pyongyang of the Democratic Republic of Korea. KMC's constant endeavor is to enhance its interaction with SAARC countries, other International agencies and many other major cities of the world to achieve better urban management and developmental programs for Kathmandu.

targets: answer: Kathmandu Metropolitan City
```

In [35]:
# UNIT TEST
w3_unittest.test_parse_squad(parse_squad)

 All tests passed


Bạn sẽ sử dụng 40000 mẫu để đào tạo và 5000 mẫu để kiểm tra

In [36]:
# 40K pairs for training
inputs_train = inputs[0:40000] 
targets_train = targets[0:40000]  

# 5K pairs for testing
inputs_test = inputs[40000:45000] 
targets_test =  targets[40000:45000] 

Bây giờ, bạn có thể tạo tập dữ liệu hàng loạt gồm các chuỗi được đệm. Trước tiên, bạn sẽ mã hóa các đầu vào và mục tiêu. Sau đó, bằng cách sử dụng hàm `tf.keras.preprocessing.sequence.pad_sequences`, bạn sẽ đảm bảo rằng đầu vào và đầu ra có độ dài cần thiết. Hãy nhớ rằng các chuỗi dài hơn kích thước yêu cầu sẽ bị cắt bớt và các chuỗi ngắn hơn sẽ được đệm bằng `0`. Thiết lập này rất giống với thiết lập khác được sử dụng trong sổ ghi chép này và sổ ghi chép trước đó.

In [37]:
# Limit the size of the input and output data so this can run in this environment
encoder_maxlen = 150
decoder_maxlen = 50

inputs_str = [tokenizer.tokenize(s) for s in inputs_train]
targets_str = [tf.concat([tokenizer.tokenize(s), [1]], 0) for s in targets_train]

inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs_str, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets_str, maxlen=decoder_maxlen, padding='post', truncating='post')

inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

# Create the final training dataset.
BUFFER_SIZE = 10000
BATCH_SIZE = 64
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

<a name='3-2'></a>
### 3.2 Tinh chỉnh mẫu T5

Bây giờ, bạn sẽ huấn luyện mô hình trong 2 kỷ nguyên. Trong mô hình T5, tất cả trọng số đều được điều chỉnh trong quá trình tinh chỉnh. Như thường lệ, việc tinh chỉnh mô hình này để đạt được kết quả hiện đại sẽ cần nhiều thời gian và tài nguyên hơn mức hiện có trong môi trường này. Tuy nhiên, bạn có thể đào tạo mô hình cho nhiều kỷ nguyên hơn và có nhiều dữ liệu hơn bằng cách sử dụng GPU Colab.

In [ ]:
# Define the number of epochs
epochs = 2
losses = []

# Training loop
for epoch in range(epochs):
    
    start = time.time()
    train_loss.reset_states()
    number_of_batches=len(list(enumerate(dataset)))

    for (batch, (inp, tar)) in enumerate(dataset):
        print(f'Epoch {epoch+1}, Batch {batch+1}/{number_of_batches}', end='\r')
        transformer_utils.train_step(inp, tar, transformer, loss_object, optimizer, train_loss)
    
    print (f'Epoch {epoch+1}, Loss {train_loss.result():.4f}')
    losses.append(train_loss.result())
    
    print (f'Time taken for one epoch: {time.time() - start} sec')
    #if epoch % 15 == 0:
        #transformer.save_weights('./pretrained_models/model_qa_temp')
# Save the final model
#transformer.save_weights('./pretrained_models/model_qa_temp')

Để có được một mô hình hoạt động bình thường, bạn cần đào tạo trong khoảng 100 kỷ nguyên. Vì vậy, chúng tôi đã đào tạo trước một mô hình cho bạn. Chỉ cần tải trọng số trong mô hình hiện tại và sử dụng nó để trả lời các câu hỏi

In [38]:
# Restore the weights
transformer.load_weights('./pretrained_models/model_qa3')

<a name='3-3'></a>
### 3.3 - Triển khai mô hình Trả lời câu hỏi của bạn
Ở bước cuối cùng này, bạn sẽ triển khai hàm answer_question, sử dụng mô hình máy biến áp được huấn luyện trước để trả lời câu hỏi.

Để giúp bạn sử dụng chức năng `transformer_utils.next_word` được cung cấp. Hàm này nhận câu hỏi và phần mở đầu của câu trả lời (cả ở định dạng tensor) cùng với mô hình để dự đoán mã thông báo tiếp theo trong câu trả lời. Ô tiếp theo hiển thị cách sử dụng:

In [39]:
# Define an example question
example_question = "question: What color is the sky? context: Sky is blue"

# Question is tokenized and padded
# Note that this is hardcoded here but you must implement this in the upcoming exercise
tokenized_padded_question = tf.constant([[822, 10, 363, 945, 19, 8, 5796, 58, 2625, 10, 5643, 19, 1692, 0, 0]])

# All answers begin with the string "answer: "
# Feel free to check that this is indeed the tokenized version of that string
tokenized_answer = tf.constant([[1525,   10]])

# Predict the next word using the transformer_utils.next_word function
# Notice that it expects the question, answer and model (in that order)
next_word = transformer_utils.next_word(tokenized_padded_question, tokenized_answer, transformer)

print(f"Predicted next word is: '{tokenizer.detokenize(next_word).numpy()[0].decode('utf-8')}'")

# Concatenate predicted word with answer so far
answer_so_far = tf.concat([tokenized_answer, next_word], axis=-1)

print(f"Answer so far: '{tokenizer.detokenize(answer_so_far).numpy()[0].decode('utf-8')}'")

Predicted next word is: 'blue'
Answer so far: 'answer: blue'


<a name='ex-3'></a>
### Bài tập 3 - Thực hiện chức năng trả lời câu hỏi

Triển khai hàm `answer_question`. Dưới đây là các bước:
- **Thiết lập câu hỏi:**

- Mã hóa câu hỏi đã cho bằng cách sử dụng mã thông báo được cung cấp.
- Thêm một chiều bổ sung cho tensor để tương thích.
- Đệm tensor câu hỏi bằng cách sử dụng `pad_sequences` để đảm bảo chuỗi có độ dài tối đa được chỉ định. Hàm này sẽ cắt ngắn chuỗi nếu nó lớn hơn hoặc đệm bằng số 0 nếu nó ngắn hơn.
- **Thiết lập câu trả lời:**
- Token hóa câu trả lời ban đầu, lưu ý tất cả các câu trả lời đều bắt đầu bằng chuỗi “answer:”.
- Thêm một chiều bổ sung cho tensor để tương thích.
- Lấy id của token `EOS`, thường được biểu thị bằng 1.
- **Tạo câu trả lời:**
- Vòng lặp cho các lần lặp `decode_maxlen`.
- Sử dụng hàm `transformer_utils.next_word` để dự đoán mã thông báo tiếp theo trong câu trả lời bằng cách sử dụng mô hình, tài liệu đầu vào và trạng thái hiện tại của đầu ra.
- Nối từ tiếp theo được dự đoán với tensor đầu ra.
- **Điều kiện dừng:**
- Quá trình tạo văn bản sẽ dừng nếu mô hình dự đoán mã thông báo `EOS`.
- Nếu dự đoán được mã thông báo `EOS`, hãy thoát khỏi vòng lặp.

In [43]:
# GRADED FUNCTION: answer_question
def answer_question(question, model, tokenizer, encoder_maxlen=150, decoder_maxlen=50):
    """
    A function for question answering using the transformer model
    Arguments:
        question (tf.Tensor): Input data with question and context
        model (tf.keras.model): The transformer model
        tokenizer (function): The SentencePiece tokenizer
        encoder_maxlen (number): Max length of the encoded sequence
        decoder_maxlen (number): Max length of the decoded sequence
    Returns:
        _ (str): The answer to the question
    """
    
    ### START CODE HERE ###
    
    # QUESTION SETUP
    
    # Tokenize the question
    tokenized_question = tokenizer.tokenize(question)
    
    # Add an extra dimension to the tensor
    tokenized_question = tf.expand_dims(tokenized_question, 0) 
    
    # Pad the question tensor
    padded_question = tf.keras.preprocessing.sequence.pad_sequences(tokenized_question,
                                                                    maxlen=encoder_maxlen,
                                                                    padding='post', 
                                                                    truncating='post') 
    # ANSWER SETUP
    
    # Tokenize the answer
    # Hint: All answers begin with the string "answer: "
    tokenized_answer = tokenizer.tokenize('answer: ')
    
    # Add an extra dimension to the tensor
    tokenized_answer = tf.expand_dims(tokenized_answer, 0)
    
    # Get the id of the EOS token
    eos = tokenizer.string_to_id("</s>") 
    
    # Loop for decoder_maxlen iterations
    for i in range(decoder_maxlen):
        
        # Predict the next word using the model, the input document and the current state of output
        next_word = transformer_utils.next_word(padded_question, tokenized_answer, model)
        
        # Concat the predicted next word to the output 
        tokenized_answer = tf.concat([tokenized_answer, next_word], axis=1)
        
        # The text generation stops if the model predicts the EOS token
        if next_word.numpy()[0][0] == eos:
            break 
    
    ### END CODE HERE ###

    return tokenized_answer 

Hãy kiểm tra mô hình với một số câu hỏi từ tập dữ liệu huấn luyện. Kiểm tra xem các câu trả lời có khớp với câu trả lời đúng không.

In [44]:
idx = 10408

result = answer_question(inputs_train[idx], transformer, tokenizer)
print(colored(pretty_decode(result, sentinels, tokenizer).numpy()[0], 'blue'))
print()
print(inputs_train[idx])
print(colored(targets_train[idx], 'green'))

b'answer: January 9, 1957'

question: When was the Chechen-Ingush Autonomous Soviet Socialist Republic transferred from the Georgian SSR? context: On January 9, 1957, Karachay Autonomous Oblast and Chechen-Ingush Autonomous Soviet Socialist Republic were restored by Khrushchev and they were transferred from the Georgian SSR back to the Russian SFSR.
answer: January 9, 1957


#### **Đầu ra dự kiến:**
```
b'answer: January 9, 1957'

question: When was the Chechen-Ingush Autonomous Soviet Socialist Republic transferred from the Georgian SSR? context: On January 9, 1957, Karachay Autonomous Oblast and Chechen-Ingush Autonomous Soviet Socialist Republic were restored by Khrushchev and they were transferred from the Georgian SSR back to the Russian SFSR.
answer: January 9, 1957
```

In [45]:
# UNIT TEST
w3_unittest.test_answer_question(answer_question)

 All tests passed


Kiểm tra mô hình với câu hỏi 110

In [46]:
idx = 110
result = answer_question(inputs_test[idx], transformer, tokenizer)
print(colored(pretty_decode(result, sentinels, tokenizer).numpy()[0], 'blue'))
print()
print(inputs_test[idx])
print(colored(targets_test[idx], 'green'))

b'answer: 50'

question:  What percentage of the vote was recorded as approving Napoleon's constitution? context: Napoleon established a political system that historian Martyn Lyons called "dictatorship by plebiscite." Worried by the democratic forces unleashed by the Revolution, but unwilling to ignore them entirely, Napoleon resorted to regular electoral consultations with the French people on his road to imperial power. He drafted the Constitution of the Year VIII and secured his own election as First Consul, taking up residence at the Tuileries. The constitution was approved in a rigged plebiscite held the following January, with 99.94 percent officially listed as voting "yes." Napoleon's brother, Lucien, had falsified the returns to show that 3 million people had participated in the plebiscite; the real number was 1.5 million. Political observers at the time assumed the eligible French voting public numbered about 5 million people, so the regime artificially doubled the participat

Kiểm tra mô hình với câu hỏi 301. Sử dụng ô này để thử nghiệm mô hình bằng cách chọn các câu hỏi kiểm tra khác. Hãy xem liệu mô hình đã học được điều gì hay nó chỉ tạo ra văn bản ngẫu nhiên.

In [47]:
idx = 311
result = answer_question(inputs_test[idx], transformer, tokenizer)
print(colored(pretty_decode(result, sentinels, tokenizer).numpy()[0], 'blue'))
print()
print(inputs_test[idx])
print(colored(targets_test[idx], 'green'))

b'answer: June 1840'

question:  On what date was a state funeral held for Napoleon? context: In 1840, Louis Philippe I obtained permission from the British to return Napoleon's remains to France. On 15 December 1840, a state funeral was held. The hearse proceeded from the Arc de Triomphe down the Champs-Élysées, across the Place de la Concorde to the Esplanade des Invalides and then to the cupola in St Jérôme's Chapel, where it remained until the tomb designed by Louis Visconti was completed. In 1861, Napoleon's remains were entombed in a porphyry sarcophagus in the crypt under the dome at Les Invalides.
answer: 15 December 1840


Xin chúc mừng, bạn đã hoàn thành nhiệm vụ cuối cùng của chuyên ngành này. Bây giờ, bạn đã biết điều gì đằng sau những mô hình mạnh mẽ như ChatGPT. Bây giờ là lúc để bạn tìm và giải quyết số lượng lớn các vấn đề có thể gặp phải với NLP.